# Stacking often is not optimal, but that's OK because projections can't really be optimized

This notebook shows how stacking often is not optimal when projections are correct (or when looking at 'perfect' lineups). However, stacking makes sense given the inaccuracies that plague even the best projections.


In [ ]:
from pprint import pprint

import pandas as pd
import pulp

In [1]:
def optimize(df, stack=None):
    """Optimizes lineup

    Args:
        df (pd.DataFrame)
        stack (int): number of players to stack with QB, default None

    Returns:
        pd.DataFrame

    """
    prob = pulp.LpProblem('FPOpt', pulp.LpMaximize)
    player_vars = pulp.LpVariable.dicts('Player', df.full_name, cat='Binary')
    prob += pulp.lpSum([pts * pvar for pvar, pts in zip(player_vars.values(), df.fppg)])
    prob += pulp.lpSum([sal * pvar for pvar, sal in zip(player_vars.values(), df.salary)]) <= 50000
    prob += pulp.lpSum(player_vars.values()) == 9
    prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                    if pos == 'QB']) == 1
    prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                        if pos == 'RB']) >= 2
    prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                        if pos == 'RB']) <= 3
    prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                        if pos == 'WR']) >= 3
    prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                        if pos == 'WR']) <= 4
    prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                        if pos == 'TE']) >= 1
    prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                        if pos == 'TE']) <= 2
    prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                        if pos == 'DST']) == 1
    if stack:
        for row in df.loc[df.pos == 'QB', :].itertuples(index=False):
            prob += pulp.lpSum([v for k,v in player_vars.items() if
                df.loc[df.full_name == k, 'team'].values[0] == row.team and
                        df.loc[df.full_name == k, 'pos'].values[0] == 'WR'] +
                        [-stack * player_vars[row.full_name]]) >= 0  

    prob.solve()
    chosen = [v.name.replace('Player_', '') for v in prob.variables() 
              if v.varValue == 1]
    return df.loc[df.full_name.isin(chosen), :]